In [57]:
import pandas as pd
import numpy as np
listings_df = pd.read_csv('./data/with_amenities_and_estimated_income.csv', low_memory = False)
#listings_df.columns.values
missing_values_count = listings_df.isnull().sum()
missing_values_count

listings_df = listings_df[np.isfinite(listings_df['estimated_income_per_month'])]

numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']

listings_df = listings_df._get_numeric_data()

#thresh = len(listings_df) * .95
#listings_df = listings_df.dropna(thresh = thresh, axis = 1, inplace = True)

listings_df = listings_df[listings_df.columns[listings_df.isnull().mean() < 0.00005]]
listings_df = listings_df.dropna(how='any',axis=0) 
#missing_values_count = listings_df.isnull().sum()
#missing_values_count
#len(listings)


In [58]:
from sklearn.model_selection import train_test_split # splitting data

train_features, test_features, train_outcome, test_outcome = train_test_split(
    listings_df.drop(["estimated_income_per_month"], axis=1),
    listings_df['estimated_income_per_month'],
    test_size=0.30, 
    random_state=11
)



In [59]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import MinMaxScaler       # scaling data
from sklearn.model_selection import GridSearchCV     # for grid search
from sklearn.pipeline import make_pipeline           # for making pipelines
from sklearn.feature_selection import SelectPercentile
from sklearn.feature_selection import VarianceThreshold
from sklearn.preprocessing import PolynomialFeatures
from sklearn.model_selection import KFold

# define a pipeline
pipe = make_pipeline(VarianceThreshold(.1), MinMaxScaler(), SelectPercentile(), KNeighborsClassifier())
folds = KFold(n_splits = 10, shuffle = True)

# defines a grid to search through
param_grid = {
              'selectpercentile__percentile':range(10, 30, 5),
              'kneighborsclassifier__n_neighbors':range(1, 10),
              'kneighborsclassifier__weights':["uniform", "distance"]
             }

# performs a grid search of pipeline
knngrid = GridSearchCV(pipe, param_grid,cv = folds, scoring="neg_mean_absolute_error")
knniqwholemodel = knngrid.fit(test_features, test_outcome)


In [63]:
features = knngrid.named_steps['feat']
listings_df.columns[features.transform(np.arange(len(listings_df.columns)))]

AttributeError: 'GridSearchCV' object has no attribute 'named_steps'

In [65]:
features = knniqwholemodel.named_steps['feat']
listings_df.columns[features.get_support()]


AttributeError: 'GridSearchCV' object has no attribute 'named_steps'

In [72]:

knngrid.cv_results_['params'][search.best_index_]

NameError: name 'search' is not defined

In [94]:
pipe.steps[3]

AttributeError: 'KNeighborsClassifier' object has no attribute 'feature_importance'

In [95]:
from sklearn.ensemble import RandomForestClassifier

pipe = make_pipeline(PolynomialFeatures(), VarianceThreshold(.1), MinMaxScaler(), RandomForestClassifier())
folds = KFold(n_splits = 10, shuffle = True)

param_grid = {
              'polynomialfeatures__degree':range(1, 3),
              'randomforestclassifier__max_depth': range(1,4),
              'randomforestclassifier__min_samples_split': range(2,5),
             }

rfcgrid = GridSearchCV(pipe, param_grid, cv = folds, scoring="neg_mean_absolute_error")
rfcwholemodel = rfcgrid.fit(test_features, test_outcome)




In [101]:
rfcgrid.best_estimator_.feature_importance()

AttributeError: 'Pipeline' object has no attribute 'feature_importance'

In [105]:
rfcwholemodel.best_estimator_.feature

AttributeError: 'Pipeline' object has no attribute 'feature_importances_'